Funcion para generar megapixeles a partir de superpixeles

In [2]:
def mas_superpixeles (segmentacion1,segmentacion2,segmentacion3,segmentacion4):
    """Función que genera una union de varias imagenes segmentadas, generando mas segmentos"""
    
    import numpy as np
    import pandas as pd
    
    #falta comprobar que todas las segmentaciones sean iguales
    
    (alto,ancho)=segmentacion1.shape #tomamos el tamaño de la imagen de entrada
    nueva_seg={}
    n_segmentacion=np.zeros((alto,ancho))
    init_seg=0
        
    for f in range(alto):
        for c in range(ancho):
            n_key=(segmentacion1[f][c],segmentacion2[f][c],segmentacion3[f][c],segmentacion4[f][c])
            #print("Nueva clave: ",n_key)
            if n_key in nueva_seg.keys():
                #print ("La clave, esta en la lista: ",nueva_seg.keys())
                n_segmentacion[f][c]=nueva_seg[n_key]
                #print ("En la nueva seg elemento ",f,c,"valor: ",nueva_seg[n_key])
            else:
                #print ("La clave, NO esta en la lista: ",nueva_seg.keys())
                nueva_seg[n_key]=init_seg
                n_segmentacion[f][c]=nueva_seg[n_key]
                #print ("En la nueva seg elemento ",f,c,"valor: ",nueva_seg[n_key])
                init_seg+=1
                #print ("nuevo el: ",init_seg)
            
                
    return n_segmentacion
    
    
    
    

Funcion eliminar los segmentos con solo un pixel de ancho

In [4]:
def elimino1 (segmentacion):

    from collections import Counter
    import numpy as np
    
    feliminar2=np.copy(segmentacion)
    print("Tamaño:", feliminar2.shape)

    frec={}
       
    for f in range(feliminar2.shape[0]-1):
        #print("f: ", f," de ",feliminar2.shape[0]-1)
        for c in range(feliminar2.shape[1]-1):
            #print("c: ", c," de ",feliminar2.shape[1]-1)
            vecinos=[]
            if f==0 and c==0 or f==len(feliminar2)-1 and c==len(feliminar2)-1 : pass
            elif f==0 and c!=len(feliminar2)-1:
                vecinos=(feliminar2[f,c-1],feliminar2[f,c],feliminar2[f,c+1],
                         feliminar2[f+1,c-1],feliminar2[f+1,c],feliminar2[f+1,c+1])
                frec=Counter(vecinos)
                if frec[feliminar2[f,c]]<=3:
                    feliminar2[f,c]=max(frec,key=frec.get)
                else:pass
            elif c==0 and f!=len(feliminar2)-1:
                vecinos=(feliminar2[f-1,c],feliminar2[f-1,c+1],
                         feliminar2[f,c],feliminar2[f,c+1],
                         feliminar2[f+1,c],feliminar2[f+1,c+1])
                frec=Counter(vecinos)
                if frec[feliminar2[f,c]]<=3:
                    feliminar2[f,c]=max(frec,key=frec.get)
                else:pass
            elif f==len(feliminar2)-1 and c!=len(feliminar2)-1:
                vecinos=(feliminar2[f-1,c-1],feliminar2[f-1,c],feliminar2[f-1,c+1],
                         feliminar2[f,c-1],feliminar2[f,c],feliminar2[f,c+1])
                frec=Counter(vecinos)
                if frec[feliminar2[f,c]]<=3:
                    feliminar2[f,c]=max(frec,key=frec.get)
                else:pass
            elif c==len(feliminar2)-1:
                vecinos=(feliminar2[f-1,c-1],feliminar2[f-1,c],
                         feliminar2[f,c-1],feliminar2[f,c],
                         feliminar2[f+1,c-1],feliminar2[f+1,c])
                frec=Counter(vecinos)
                if frec[feliminar2[f,c]]<=3:
                    feliminar2[f,c]=max(frec,key=frec.get)
                else:pass
            else:
                vecinos=(feliminar2[f-1,c-1],feliminar2[f-1,c],feliminar2[f-1,c+1],
                         feliminar2[f,c-1],feliminar2[f,c],feliminar2[f,c+1],
                         feliminar2[f+1,c-1],feliminar2[f+1,c],feliminar2[f+1,c+1])
                frec=Counter(vecinos)
                if frec[feliminar2[f,c]]<=3:
                    feliminar2[f,c]=max(frec,key=frec.get)
                else:pass

    return feliminar2


Funcion unir los segmentos de una segmentacion en función de una clusterización dada

In [ ]:
def une_segments (segmentacion,clusters):
    
    import numpy as np

    (alto,ancho)=segmentacion.shape #tomamos el tamaño de la imagen de entrada

    #n_mega=nueva_union.copy()
    n_mega=segmentacion.copy()    
    
    for f in range(alto):
        for c in range(ancho):
            if f==0:
                if c==0: #primer pixel
                    pass
                else: #primera fila
                    #print ("valor pixel: ",n_mega[f][c], "valor pixel anterior: ",n_mega[f][c-1])
                    #print ("grupo superpixel actual: ",clusters[n_segments[f][c]],"grupo superpixel anterior: ",clusters[n_segments[f][c-1]])
                    if n_mega[f][c]!=n_mega[f][c-1] : #cuando el superpixel cambia con el anterior
                        #print ("valor pixel: ",n_mega[f][c], "valor pixel anterior: ",n_mega[f][c-1],"punto: ",f,c)
                        #print ("grupo superpixel actual: ",clusters[n_segments[f][c]],"grupo superpixel anterior: ",clusters[n_segments[f][c-1]])
                        if clusters[segmentacion[f][c-1]]==clusters[segmentacion[f][c]]:# superpixel similares
                            #print("asigno grupo: ",n_mega[f][c-1], "al antiguo: ",n_mega[f][c])
                            #print(n_mega)
                            n_mega=np.where(n_mega==n_mega[f][c],n_mega[f][c-1],n_mega)
                            #print("Despues:",n_mega)
                            #print("nuevo valor del punto: ",f,c,"es: ",n_mega[f][c])
                        else:pass  

            elif c==0: #primera columna
                if n_mega[f][c]!=n_mega[f-1][c] :  #si el superpixel cambia con el superior
                    if clusters[segmentacion[f-1][c]]==clusters[segmentacion[f][c]]:# superpixel similares
                        #print ("valor pixel: ",n_mega[f][c], "valor pixel anterior: ",n_mega[f][c-1],"punto: ",f,c)
                        #print ("grupo superpixel actual: ",clusters[n_segments[f][c]],"grupo superpixel anterior: ",clusters[n_segments[f][c-1]])
                        #print(n_mega)
                        n_mega=np.where(n_mega==n_mega[f][c],n_mega[f][c-1],n_mega)
                        #print("Despues:",n_mega)
                    else:pass
                    
                else:pass
        
            else: #otro punto normal
                if n_mega[f][c]!=n_mega[f][c-1] : #cuando el superpixel cambia con el anterior
                    if clusters[segmentacion[f][c-1]]==clusters[segmentacion[f][c]]:# superpixel similares
                        #print ("valor pixel: ",n_mega[f][c], "valor pixel anterior: ",n_mega[f][c-1],"punto: ",f,c)
                        #print ("grupo superpixel actual: ",clusters[n_segments[f][c]],"grupo superpixel anterior: ",clusters[n_segments[f][c-1]])
                        #print(n_mega)
                        n_mega=np.where(n_mega==n_mega[f][c],n_mega[f][c-1],n_mega)
                        #print("Despues:",n_mega)
                    else:pass
                    
                elif n_mega[f][c]!=n_mega[f-1][c] :  #si el superpixel cambia con el superior
                    if clusters[segmentacion[f-1][c]]==clusters[segmentacion[f][c]]:# superpixel similares
                        #print ("valor pixel: ",n_mega[f][c], "valor pixel anterior: ",n_mega[f][c-1],"punto: ",f,c)
                        #print ("grupo superpixel actual: ",clusters[n_segments[f][c]],"grupo superpixel anterior: ",clusters[n_segments[f][c-1]])
                        #print(n_mega)
                        n_mega=np.where(n_mega==n_mega[f][c],n_mega[f][c-1],n_mega)
                        #print("Despues:",n_mega)
                    else:
                        pass
                else:
                    pass
                    
    return n_mega

Funcion eliminar de una segmentación los segmentos menores que un tamaño determinado en %

In [1]:
def une_segments2(segmentacion,distancias,clusters):
    """Dada una segmentación, une los elementos mas parecidos de acuerdo con la clusterizacion"""
    
    import cv2
    import numpy as np
    
    vecinos=[]
    distancias_rel=[]
    modulo=[]
    
    #cojo los difernentes segmentos generados"
    elem=np.unique(segmentacion)
    print("segmentos: ", elem)
    
    #calculo el tamaño de cada segmentacion y el total, identificacion
    
    total=segmentacion.shape[0]*segmentacion.shape[1]
    #print("pixeles totales: ", total)
    
    for i in elem:
     
        # Crear un kernel para la dilatación (en este caso, un cuadro 3x3)
        kernel = np.ones((3, 3), np.uint8)

        # Aplicar la dilatación al segmento
        aux=(segmentacion==i)
        aux=aux.astype(np.uint8)
        #print(type(aux))
        #print("segmento trabajo: ",i,"Tamaño inicial: ",np.sum(aux==1))
        aux_dilatado= cv2.dilate(aux, kernel, iterations=1)
        #print("Tamaño final: ",np.sum(aux_dilatado==1))

        #busco los vecinos sobre la segmentacion con la mascara dilatada

        vecinos=np.unique(np.where(aux_dilatado==1,segmentacion,i))
        print("vector de vecinos: ",vecinos)

        for v in vecinos:
            #print("segmento: ",i,"caracteristicas: ", distancias[i])
            #print("segmento: ",dato[0])
            #print("caracteristicas: ", distancias[dato[0]])
            #print("segmento vecino: ",v,"caracteristicas: ", distancias[v])
            dist=distancias[i] - distancias[v]

            if v==i: modulo.append(np.inf)
            elif v in elem_borde: modulo.append(np.inf)
            else: modulo.append(np.linalg.norm(dist))
            #print("REL: ", distancias_rel,"valor: ",modulo)

        parecido=vecinos[np.argmin(modulo)]
        #print("el mas parecido es: ",parecido)

        #sustituyo el segmento por el mas parecido

        #print("Segmento eliminado: ",i," sustituido por: ",parecido, "puntos anteriores: ", np.sum(segmentacion==i))
        if clusters[i]==clusters[parecido]:
            print("cluster pixel: ",clusters[i], "cluster mejor vecino: ",clusters[parecido])
            segmentacion=np.where(segmentacion==i,parecido,segmentacion)
        #print("El segmento: ",i, "queda: ",np.sum(segmentacion==i))

        modulo=[] 
        #print("Nuevos segmentos= ",np.unique(segmentacion))
            
    
    return segmentacion

In [ ]:
def quita_pequenos(img_original,segmentacion,distancias,tam=1):
    """quitar los grupos con un tamaño inferor al establecido en %"""
    
    import cv2
    import numpy as np
    
    vecinos=[]
    distancias_rel=[]
    modulo=[]
    
    #cojo los difernentes segmentos generados"
    elem=np.unique(segmentacion)
#     print("segmentos: ", elem)
    
    #calculo el tamaño de cada segmentacion y el total, identificacion
    
    total=segmentacion.shape[0]*segmentacion.shape[1]
    #print("pixeles totales: ", total)
    tamano=[]
    for i in elem:
        aux=(segmentacion==i)
        parcial=np.sum(aux==1)
        #print(parcial)
        if parcial==0:pass
        else:
            porcentaje=round((parcial/total)*100,4)
            #print(porcentaje)
            if porcentaje<tam:
                #print("mascara :",i)
                #print("suma parcial: ",parcial, " porcentaje: ",porcentaje,"%")
                #print(tamano)
                
                # Crear un kernel para la dilatación (en este caso, un cuadro 3x3)
                kernel = np.ones((3, 3), np.uint8)
                
                # Aplicar la dilatación al segmento
        
                aux=(segmentacion==i)
                aux=aux.astype(np.uint8)
                #print(type(aux))
                #print("segmento trabajo: ",i,"Tamaño inicial: ",np.sum(aux==1))
                aux_dilatado= cv2.dilate(aux, kernel, iterations=1)
                #print("Tamaño final: ",np.sum(aux_dilatado==1))
                
                #busco los vecinos sobre la segmentacion con la mascara dilatada
                
                vecinos=np.unique(np.where(aux_dilatado==1,segmentacion,i))
                #print("vector de vecinos: ",vecinos)
                
                 #busco el vecino mas cercano
                for v in vecinos:
                    #print("segmento: ",i,"caracteristicas: ", distancias[i])
                    #print("segmento: ",dato[0])
                    #print("caracteristicas: ", distancias[dato[0]])
                    #print("segmento vecino: ",v,"caracteristicas: ", distancias[v])
                    dist=distancias[i] - distancias[v]
                    distancias_rel.append(dist)
                    if v==i: modulo.append(np.inf)
                    else: modulo.append(np.linalg.norm(dist))
                    #print("REL: ", distancias_rel,"valor: ",modulo)
                    parecido=vecinos[np.argmin(modulo)]
                    #print("el mas parecido es: ",parecido)
                    
                #sustituyo el segmento por el mas parecido
                    
                #print("Segmento eliminado: ",i," sustituido por: ",parecido, "puntos anteriores: ", np.sum(segmentacion==i))
                segmentacion=np.where(segmentacion==i,parecido,segmentacion)
                #print("El segmento: ",i, "queda: ",np.sum(segmentacion==i))
                distancias_rel=[]
                modulo=[] 
                #print("Nuevos segmentos= ",np.unique(segmentacion))
    
    return segmentacion
        
    

Funcion reordenar segmentos


In [1]:
def reordenar (segmentacion):
    """Funcion que renombra los segmentos para que no haya saltos en la numeracion entre ellos"""
    import numpy as np
    
    valores=list(np.unique(segmentacion))
    #print("Los segmentos que hay son: ", valores)
    
    # recorro los segmentos y le doy el valor de posición en el vector valores.
    for elem in valores:
        segmentacion=np.where(segmentacion==elem, valores.index(elem)+1, segmentacion)
    
    #print("Lo que queda al final es: ", np.unique(segmentacion))

    return segmentacion
    